In [3]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "SNHU1234"
host = 'nv-desktop-services.apporto.com'
port = 30351
database = "AAC"
collection = "animals"

# Connect to database via CRUD Module
db = AnimalShelter(username, password, host, port, database, collection)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = '/home/dylancavazos_snhu/Pictures/Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Grazioso Salvare Dashboard'))),
    html.Center(
        html.A(
            html.Img(src='data:image/png;base64,{}'.format(encoded_image), style={'width': '10%'}),
            href='http://snhu.edu'
        )
    ),
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.


    html.Center(html.B(html.P('Dashboard by Dylan Cavazos'))),
    html.Hr(),
    html.Div([
        # Dropdown | Dash for Python Documentation | Plotly. (n.d.). https://dash.plotly.com/dash-core-components/dropdown
        dcc.Dropdown(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'Water'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain'},
            {'label': 'Disaster Rescue or Individual Tracking', 'value': 'Disaster'},
            {'label': 'Reset', 'value': 'All'}
        ],
        placeholder="Select a filter",
        value='All'
        )
    ]),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current=0,
        page_size=10
    ),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    if filter_type == 'Water':
        query = {
            "animal_type": "Dog", 
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif filter_type == 'Mountain':
        query = {
            "animal_type": "Dog",
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif filter_type == 'Disaster':
        query = {
            "animal_type": "Dog", 
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }
    else:
        query = {}
    
    print("Executing query:", query)
    result = db.read(query)
    print("Query result:", result)
        
    df = pd.DataFrame.from_records(db.read(query))
    if '_id' in df.columns:
        df.drop(columns=['_id'], inplace=True)
    return df.to_dict('records')

## FIX ME Add code to filter interactive data table with MongoDB queries
#
#        
#        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
#        data=df.to_dict('records')
#       
#       
#        return (data,columns)

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)

def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    print("DataFrame Columns: ", dff.columns)
    if 'breed' in dff.columns:
        fig = px.pie(dff, names='breed', title='Preferred Animals')
        return [
            dcc.Graph(figure=fig)
        ]
    else:
        return [
            html.P("No breed data available to display.")
        ]
    

    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #

    #return [
    #    dcc.Graph(            
    #        figure = px.pie(df, names='breed', title='Preferred Animals')
    #    )    
    #]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)

def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)


def update_map(viewData, index):  
    if viewData is None or index is None or len(index) == 0:
        return []
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    row = index[0]
    
    if len(dff.columns) < 15:
        return [
            html.P("Not enough data to display map.")
        ]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True, port=8052)


Connection to MongoDB successful.
Dash app running on http://127.0.0.1:8052/
DataFrame Columns:  Index([], dtype='object')
Executing query: {}


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Executing query: {}


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



DataFrame Columns:  Index(['rec_num', 'age_upon_outcome', 'animal_id', 'animal_type', 'breed',
       'color', 'date_of_birth', 'datetime', 'monthyear', 'name',
       'outcome_subtype', 'outcome_type', 'sex_upon_outcome', 'location_lat',
       'location_long', 'age_upon_outcome_in_weeks', ''],
      dtype='object')
Executing query: {'animal_type': 'Dog', 'breed': {'$in': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']}, 'sex_upon_outcome': 'Intact Female', 'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}}
Query result: [{'_id': ObjectId('664e861a9ce8e9b6ba568ac6'), 'rec_num': 36, 'age_upon_outcome': '6 months', 'animal_id': 'A706953', 'animal_type': 'Dog', 'breed': 'Labrador Retriever Mix', 'color': 'Yellow', 'date_of_birth': '2014-12-06', 'datetime': '2015-07-06 11:33:00', 'monthyear': '2015-07-06T11:33:00', 'name': '', 'outcome_subtype': 'Medical', 'outcome_type': 'Euthanasia', 'sex_upon_outcome': 'Intact Female', 'location_lat': 30.5480802368633, 'loca

DataFrame Columns:  Index(['rec_num', 'age_upon_outcome', 'animal_id', 'animal_type', 'breed',
       'color', 'date_of_birth', 'datetime', 'monthyear', 'name',
       'outcome_subtype', 'outcome_type', 'sex_upon_outcome', 'location_lat',
       'location_long', 'age_upon_outcome_in_weeks', ''],
      dtype='object')
Executing query: {'animal_type': 'Dog', 'breed': {'$in': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']}, 'sex_upon_outcome': 'Intact Male', 'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}}
Query result: [{'_id': ObjectId('664e861a9ce8e9b6ba569f53'), 'rec_num': 5315, 'age_upon_outcome': '2 years', 'animal_id': 'A708726', 'animal_type': 'Dog', 'breed': 'Alaskan Malamute', 'color': 'Sable/White', 'date_of_birth': '2013-07-30', 'datetime': '2015-08-02 17:24:00', 'monthyear': '2015-08-02T17:24:00', 'name': 'Papa', 'outcome_subtype': '', 'outcome_type': 'Return to Owner', 'sex_upon_outcome': 'Intact Male', 'location_lat': 

Executing query: {'animal_type': 'Dog', 'breed': {'$in': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']}, 'sex_upon_outcome': 'Intact Male', 'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}}
Query result: [{'_id': ObjectId('664e861a9ce8e9b6ba569f53'), 'rec_num': 5315, 'age_upon_outcome': '2 years', 'animal_id': 'A708726', 'animal_type': 'Dog', 'breed': 'Alaskan Malamute', 'color': 'Sable/White', 'date_of_birth': '2013-07-30', 'datetime': '2015-08-02 17:24:00', 'monthyear': '2015-08-02T17:24:00', 'name': 'Papa', 'outcome_subtype': '', 'outcome_type': 'Return to Owner', 'sex_upon_outcome': 'Intact Male', 'location_lat': 30.4309339291938, 'location_long': -97.480825836737, 'age_upon_outcome_in_weeks': 104.817857142857}, {'_id': ObjectId('6674c69381bba2aea3d95890'), '': 5315, 'age_upon_outcome': '2 years', 'animal_id': 'A708726', 'animal_type': 'Dog', 'breed': 'Alaskan Malamute', 'color': 'Sable/White', 'date_of_birth': '2013-07-30', '

Executing query: {}


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



DataFrame Columns:  Index(['rec_num', 'age_upon_outcome', 'animal_id', 'animal_type', 'breed',
       'color', 'date_of_birth', 'datetime', 'monthyear', 'name',
       'outcome_subtype', 'outcome_type', 'sex_upon_outcome', 'location_lat',
       'location_long', 'age_upon_outcome_in_weeks', ''],
      dtype='object')
